In [1]:
import numpy as np
x_train=np.load('../../../datasets/MNIST/d_mnist40k.npy')
x_train=x_train.reshape(x_train.shape[0]*x_train.shape[1],2000)
y=np.load('../../../datasets/MNIST/l_mnist40k.npy')
y=y.reshape(y.shape[0]*y.shape[1],)
alpha_trans1=np.load('../../../datasets/MNIST/Sb_mnist40k.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32') 

In [2]:
import glob
import numpy as np
import random
import tensorflow.compat.v1 as tf
import os
from sklearn.utils import shuffle
batchSize=500
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import fractional_matrix_power
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
from tensorflow.keras import backend as B
k_cluster = 10
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
from tensorflow.keras.models import Model
import sys
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from munkres import Munkres


def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    #print('uc1 shape is',l1)
    #print('uc2 shape is',l2)
    
    ## if 10==10 return true else return false
    assert(l1 == l2 and np.all(uc1 == uc2))
    m = np.ones([l1, l2])
    for i in range(l1):
        it_i = np.nonzero(c1 == uc1[i])[0]
        for j in range(l2):
            it_j = np.nonzero(c2 == uc2[j])[0]
            m_ij = np.intersect1d(it_j, it_i)
            m[i,j] =  -m_ij.size
    return m

def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes) 
    m = Munkres()
    indexes = m.compute(cost_matrix)
    mapper = { old: new for (old, new) in indexes }
    new_labels = translate_clustering(labels, mapper)
    new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
    return new_labels,accuracy(new_cm)


def arch():
    encoding_dim = 490 
    input_img = Input(shape=(2000,))
    encoded_layer_2 = Dense(1500, activation='tanh')(input_img)
    encoded_layer_3 = Dense(1000, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(700, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(700, activation='tanh')(encoded)
    decoded_layer_3 = Dense(1000, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(1500, activation='tanh')(decoded_layer_3)
    decoded = Dense(2000, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder

def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T      
    
    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + B.epsilon())).T
    return q


In [3]:
encoder,encoding_dim,autoencoder=arch()
print('loading trained model')
encoder.load_weights('../../../checkpoints/chk_mnist/sdsc_mnist40k.ckpt')
z_test=encoder.predict(x_train)
ic= np.load('../../../checkpoints/chk_mnist/sdsc_icmnist40k.npy')
assignment = assign_func(z_test, ic)
predictedlabels = np.argmax(assignment, axis=1)
rectified_label, acc=rectify_label(predictedlabels, y)
nmi=sklearn.metrics.normalized_mutual_info_score(y, predictedlabels)
f1=sklearn.metrics.f1_score(y, rectified_label,average='micro')
prec=sklearn.metrics.precision_score(y, rectified_label,average='weighted')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
loading trained model


In [4]:
print(acc,nmi,f1,prec)

0.96775 0.9160532969450231 0.96775 0.9679329214317739


In [5]:
x=np.load('../../../datasets/MNIST/d_mnist70k.npy')
x=x.reshape(x.shape[0]*x.shape[1],2000)
y=np.load('../../../datasets/MNIST/l_mnist70k.npy')
y=y.reshape(y.shape[0]*y.shape[1],)
x_test=x[40000:,:]
y_test=y[40000:,]
z_test=encoder.predict(x_test)
assignment = assign_func(z_test, ic)
predictedlabels = np.argmax(assignment, axis=1)
rectified_label, acc=rectify_label(predictedlabels, y_test)
nmi=sklearn.metrics.normalized_mutual_info_score(y_test, predictedlabels)
f1=sklearn.metrics.f1_score(y_test, rectified_label,average='micro')
prec=sklearn.metrics.precision_score(y_test, rectified_label,average='weighted')
print(acc,nmi,f1,prec)

0.9701333333333333 0.9215328881744033 0.9701333333333333 0.9702494630546102
